In [ ]:
!pip install -q kaggle
from google.colab import drive
drive.mount('/content/gdrive')
 
!pip3 install --upgrade --force-reinstall --no-deps kaggle
 
!mkdir ~/.kaggle  
!cp ./gdrive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install tensorflow-text
!pip install -q tf-models-official==2.4.0

In [ ]:
!kaggle competitions download -c jigsaw-toxic-severity-rating
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!kaggle datasets download -d julian3833/jigsaw-unintended-bias-in-toxicity-classification

In [ ]:
!mkdir data
!mkdir data1
!mkdir data2

In [ ]:
!unzip jigsaw-toxic-severity-rating.zip -d data
!unzip jigsaw-toxic-comment-classification-challenge -d data1
!unzip jigsaw-unintended-bias-in-toxicity-classification -d data2

In [ ]:
!unzip data1/train.csv.zip -d data1

In [ ]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow_hub as hub
import tensorflow_text as text
import re
import nltk
import os
from official import nlp
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-12_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'electra_large':
        'https://tfhub.dev/google/electra_large/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    'lambert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/lambert_en_uncased_L-24_H-1024_A-16/2',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_large':
        'https://tfhub.dev/google/electra_large/2',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'lambert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

In [ ]:
stop_words = stopwords.words('english')
snowball = SnowballStemmer(language='english')

def transform(input_string : tf.Tensor):
  s = input_string.numpy().decode('utf-8')
  s = re.sub('[^A-Za-z0-9]+', ' ', s).lower().split()
  s = [i for i in s if i not in stop_words]
  s = [snowball.stem(i) for i in s]
  s = ' '.join(s)
  return s

In [ ]:
def transform_pd(input_string):
  s = re.sub('[^A-Za-z0-9]+', ' ', input_string).lower().split()
  s = [i for i in s if i not in stop_words]
  s = [snowball.stem(i) for i in s]
  s = ' '.join(s)
  return s

In [ ]:
train = pd.read_csv('data2/train.csv')

In [ ]:
data = tf.data.Dataset.from_tensor_slices((train['comment_text'], train['target']))

In [ ]:
dist = np.linspace(0, 1, num=10)
dist = np.column_stack([dist[:-1:], dist[1::]])

In [ ]:
ip = []
pl = []
for i in dist:
  ip.append(train[(train['target'] >= i[0]) & (train['target'] < i[1])].shape[0] / train.shape[0])
  pl.append(1 - train[(train['target'] >= i[0]) & (train['target'] < i[1])].shape[0] / train.shape[0])
ip = tf.convert_to_tensor(np.array(ip))
pl = np.array(pl)
pl = tf.convert_to_tensor(pl / pl.sum())
dist = tf.convert_to_tensor(dist)

In [ ]:
pl

In [ ]:
dist = np.linspace(0, 1, num=10)
dist = tf.convert_to_tensor(dist[:-1:])

In [ ]:
dist

In [ ]:
@tf.function
def find(x : tf.Tensor):
  return tf.cast(tf.reduce_sum(tf.cast(tf.less_equal(dist, x), dtype=tf.float64)) - 1, dtype=tf.int32)

In [ ]:
find(tf.constant(1, dtype=tf.float64))

In [ ]:
data = data.rejection_resample(lambda x, y: find(y), target_dist=pl).map(lambda a, b: b)

In [ ]:
data_sample = data.take(10000).map(lambda x, y: y)

In [ ]:
data_sample = list(data_sample.as_numpy_iterator())

In [ ]:
df = pd.DataFrame({'target': data_sample})

In [ ]:
df['target'].hist(bins=20)

In [ ]:
data = data.batch(32).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

In [ ]:
def get_model_v_1():
  with strategy.scope():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(200, activation=tf.nn.elu)(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(200, activation=tf.nn.elu)(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(net)
    model = tf.keras.Model(text_input, net)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='MAE', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
def get_model_v_2():
  with strategy.scope():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    encoder = hub.KerasLayer("https://tfhub.dev/google/sentence-t5/st5-3b/1", trainable=True, input_shape=[], dtype=tf.string)
    net = encoder(text_input)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(200, activation=tf.nn.elu)(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(200, activation=tf.nn.elu)(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(net)
    model = tf.keras.Model(text_input, net)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='MAE', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
model = get_model_v_2()

In [ ]:
model.fit(x=data, epochs=1, batch_size=32)

### Evaluation

In [ ]:
comments = pd.read_csv('data/comments_to_score.csv')
comments

In [ ]:
comments['transform_text'] = comments['text']

In [ ]:
test_y = model.predict(comments['transform_text'])

In [ ]:
comments['target'] = test_y

In [ ]:
test_y

In [ ]:
comments.iloc[0:7:]['transform_text']

In [ ]:
text = comments.iloc[0:7:]['transform_text']
print(text)
for layer in model.layers[0:3]:
  text = layer(text)
  print(text)

In [ ]:
text['pooled_output'][0] - text['pooled_output'][1]

In [ ]:
comments[comments['target'] > 0.5]

In [ ]:
comments.to_csv('result.csv')

In [ ]:
validation = pd.read_csv('data/validation_data.csv')
validation.iloc[0]['more_toxic']

In [ ]:
rates_less_toxic = []
rates_more_toxic = []
for i in range(len(validation)):
  less_toxic = validation.iloc[i]['less_toxic']
  more_toxic = validation.iloc[i]['more_toxic']
  rate_less_toxic = comments[comments['text'] == less_toxic]['target']
  rate_more_toxic = comments[comments['text'] == more_toxic]['target']
  if len(rate_less_toxic) > 0 and len(rate_more_toxic) > 0:
    rates_less_toxic.append(rate_less_toxic.iloc[0])
    rates_more_toxic.append(rate_more_toxic.iloc[0])

In [ ]:
rates_less_toxic = np.array(rates_less_toxic)
rates_more_toxic = np.array(rates_more_toxic)

In [ ]:
d = rates_more_toxic - rates_less_toxic

In [ ]:
df = pd.DataFrame({'d': d, 'rates_more_toxic': rates_more_toxic, 'rates_less_toxic': rates_less_toxic})

In [ ]:
df[df['rates_less_toxic'] < 0.2]

In [ ]:
df.hist()

In [ ]:
d[d > 0].shape[0] / d.shape[0]

In [ ]:
d[d >= 0].shape[0] / d.shape[0]

In [ ]:
model.save('model_1')

In [ ]:
!zip -r model_1.zip  model_1/

In [ ]:
!mv model_1.zip /content/gdrive/MyDrive/